In [37]:
import os
import re
import string
import pandas as pd
import seaborn as sns
from PIL import Image


In [39]:
cloud_clusters_enhanced = pd.read_excel(r'C:\Users\lesze\cloud_clusters_enhanced.xlsx', index_col=0)
base_path = r"C:\Users\lesze\orbitfolder\images"

In [41]:
import os
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

# Create output directory if it doesn't exist
output_dir = "cluster_crops"
os.makedirs(output_dir, exist_ok=True)

for index, row in cloud_clusters_enhanced.iterrows():
    orbit_no = index if 'orbit_no' not in row else row['orbit_no']
    cluster_name = row['cluster_name']
    cluster_cloud_type = row['cluster_cloud_type']
    x_img = row['cluster_center_x']  # Original x in full image
    y_img = row['cluster_center_y']  # Original y in full image

    prefix = f'orbit{str(orbit_no).zfill(5)}'
    
    for filename in os.listdir(base_path):
        if filename.startswith(prefix):
            try:
                file_path = os.path.join(base_path, filename)
                with Image.open(file_path) as bg_img:
                    # Convert RGBA to RGB if needed (PNG supports both)
                    if bg_img.mode == 'RGBA':
                        bg_img_rgb = bg_img.convert('RGB')
                    else:
                        bg_img_rgb = bg_img.copy()
                    
                    # Calculate crop coordinates (200x200 area for better quality)
                    crop_size = 200
                    left = max(0, x_img - crop_size//2)
                    top = max(0, y_img - crop_size//2)
                    right = min(bg_img.width, x_img + crop_size//2)
                    bottom = min(bg_img.height, y_img + crop_size//2)
                    
                    # Edge handling
                    if right - left < crop_size:
                        left = max(0, right - crop_size) if left != 0 else left
                        right = left + crop_size
                    if bottom - top < crop_size:
                        top = max(0, bottom - crop_size) if top != 0 else top
                        bottom = top + crop_size
                    
                    # Crop and convert to high-quality RGB
                    crop = bg_img_rgb.crop((left, top, right, bottom))
                    
                    # Calculate center position
                    x_center = x_img - left
                    y_center = y_img - top
                    
                    # Draw 3-pixel wide red X
                    draw = ImageDraw.Draw(crop)
                    cross_size = 5  # Length of X arms
                    line_width = 2    # Width of X lines
                    
                    # Draw two diagonal lines for X
                    draw.line([
                        (x_center-cross_size, y_center-cross_size),
                        (x_center+cross_size, y_center+cross_size)
                    ], fill='red', width=line_width)
                    
                    draw.line([
                        (x_center-cross_size, y_center+cross_size),
                        (x_center+cross_size, y_center-cross_size)
                    ], fill='red', width=line_width)
                    
                    # High-quality downscaling to 100x100
                    crop = crop.resize((100, 100), Image.LANCZOS)
                    
                    # Save as PNG (lossless)
                    safe_cloud_type = cluster_cloud_type.replace(" ", "_").replace("/", "-")
                    output_filename = f"orbit{orbit_no}_{cluster_name}_{safe_cloud_type}.png"
                    output_path = os.path.join(output_dir, output_filename)
                    crop.save(output_path, "PNG", compress_level=9)  # Balanced compression
                    
                    # Optional display
                    if SHOW_PREVIEW:  # Set this to True/False at start of script
                        plt.figure(figsize=(5,5))
                        plt.imshow(crop)
                        plt.title(f"{os.path.basename(output_path)}")
                        plt.axis('off')
                        plt.show()
                    
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")